# Creating Standard Training Data

In this notebook we will be doing the following :
  1. Build & perform basic text cleaning operations/pipeline on the documents
  2. Convert the data in training data format. i.e. _label_tag1 _label_tag2
  3. Split the dataset into : train,dev and test sets  
  4.  Divide the dataset into Groups/Classes  
    a. Dividing the entire Dataset(~1M) into 14 groups/class.  
    b. Check the Label Distribution of Labels in each 14 groups/class.  
  5. Standard practices to Create Corpus & Label Dictionary using **Flair**

**HOT TIP** : *Save them as pickle for easy rendering for experiments*



In [1]:
# First let's check what has Google given us ! Thank you Google for the GPU

!nvidia-smi

Thu Jan  9 14:02:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [10]:
# Let's mount our G-Drive. Hey !! Because for GPU you now give your data to Google 

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Install necessary packages and restart the environment

! pip install tiny-tokenizer
! pip install  flair

  Created wheel for tiny-tokenizer: filename=tiny_tokenizer-3.1.0-cp36-none-any.whl size=10550 sha256=33a3ed6bd1d88a53ee9020f40c6384bc83d41ec63e79bbda122706d4e738250d
  Stored in directory: /root/.cache/pip/wheels/d1/c8/36/334497a689fab90128232e86b5829b800dd271a3d5d5959c53
Successfully built tiny-tokenizer
     |████████████████████████████████| 194kB 2.8MB/s 
     |████████████████████████████████| 778kB 41.7MB/s 
     |████████████████████████████████| 798kB 41.2MB/s 
     |████████████████████████████████| 450kB 45.3MB/s 
     |████████████████████████████████| 1.0MB 39.5MB/s 
     |████████████████████████████████| 1.0MB 35.4MB/s 
     |████████████████████████████████| 348kB 38.7MB/s 
     |████████████████████████████████| 21.5MB 33.6MB/s 
     |████████████████████████████████| 3.8MB 33.1MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 70.7MB 132kB/s 
     |████████████████████████████████| 870kB 33.5MB/s 
  Created wheel for s

In [0]:
# Let's import our packages !

import pandas as pd
from tqdm import tqdm
import html
import re
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split
# import flair
import pickle
from torch.optim.adam import Adam
import json
# Making Corpus

from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

In [0]:
## Mentioning where is our data located on G-Drive. Make sure to rectify your path

path = '/content/drive/My Drive/ICDMAI_Tutorial/notebook/'
tag_group = '/content/drive/My Drive/ICDMAI_Tutorial/stack-overflow-tag-network/stack_network_nodes.csv'
data ='filtered_data/question_tag_text_mapping.pkl'

In [3]:
# Let's see the main Data-Set

question_tag = pd.read_pickle(path+data)
question_tag.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tag
0,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
1,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net]"
2,330,63.0,2008-08-02T02:51:36Z,NaN,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,[c++]
3,470,71.0,2008-08-02T15:11:47Z,2016-03-26T05:23:29Z,13,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,"[web-services, .net]"
4,580,91.0,2008-08-02T23:30:59Z,NaN,21,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...,[sql-server]


In [4]:
# Let's see the Groups/Classes
import pandas as pd
tag_group = pd.read_csv(tag_group)
tag_group.head()

,name,group,nodesize,group_name
0,c,1,189.83,Programming
1,c++,1,268.11,Programming
2,django,1,40.91,Programming
3,python,1,438.67,Programming
4,flask,1,9.39,Programming


## 1. Text Pre-processing Pipeline

Every try-except block can be written as a different modular function which can be invoked from preprocess_text() function. This serves as a pipeline of the series of text-cleaning that you might require for your dataset.

In [0]:
clean = re.compile('<.*?>')

def preprocess_text(text) :
  try :
    # soup = BeautifulSoup(text, "html.parser")
    # text = soup.get_text()
    text=  re.sub(clean, '', text)
    text = html.unescape(text)
  except :
    print("Error in HTML Processing ...")
    print(text)
    text = text
  try :
    # remove extra newlines (often might be present in really noisy text)
    text = text.translate(text.maketrans("\n\t\r", "   "))
  except :
    print("Error in removing extra lines ...")
    print(text)
    text = text

  try :
    # remove extra whitespace
    text = re.sub(' +', ' ', text)
    text = text.strip()
  except :
    print("Error in extra whitespace removal ...")
    print(text)
    text = text

  return text

## 2. Create Training Data Format

Here we iterate the dataset dataframe and create the format acceptable to Flair. This is a standard format for few other Text Classification models/frameworks by Facebook.

***Format***  : ____label ____**tag1** ____label ____**tag2** **text**

Here the text Document has to be in a single line which was handled in the preprocess_text() method.

In [0]:
def create_training_format(question_tag) :

  print("Preparing training data format ...")
  # training_df = pd.DataFrame("columns")
  labels = list()
  texts = list()
  for index in tqdm(question_tag.index) :
    tags = question_tag.loc[index,'Tag']
    text_label = ''
    for tag in tags :
      label = '__label__'+tag
      text_label = text_label + ' ' + label
    
    text_label = text_label.strip()
    # text =  html.unescape(question_tag.loc[index,'Body'])
    text =  question_tag.loc[index,'Title'].strip() + '. ' + question_tag.loc[index,'Body'].strip()

    # if len(text.split()) < 5 :
    #   continue 

    labels.append(text_label)
    texts.append(text)


  df = pd.DataFrame(list(zip(labels[:], texts[:])), columns =['label', 'text']) 
  # df.head()
  print("Cleaning Text ....")
  df['text'] = df['text'].apply(preprocess_text)
  print("Cleaned Data Size : {}".format(df.shape))

  return df


## 3. Create Training Splits

Here we create standard random splits of the dataset to :
  1. training set : 70 % data
  2. dev/validation set :  15 % data
  3. test set : 15 % data

#### TO DO : Experiments :
  1. Stratified Sampling of documents based on Tags
  2. Does a  70-15-15 split or 90-5-5 split make any difference when you have 1M records ?

In [0]:
def create_splits(df,path,group_id = ''):

  print("Splitting Training Data ... ")
  train_df , test_df = train_test_split(df,random_state=42,test_size=0.30)
  dev_df ,test_df = train_test_split(test_df,random_state=42,test_size=0.5)
  print("Training Dataset : {}".format(train_df.shape[0]))
  print("Validation Dataset : {}".format(dev_df.shape[0]))
  print("Test Dataset : {}".format(test_df.shape[0]))

  print("Path  : {} ".format(path+'training_data/standard/group/'+ str(group_id) + '/train.txt'))
  train_df.to_csv(path+'training_data/standard/group/'+ str(group_id) + '/train.txt',sep='\t',index=False,header=False)
  dev_df.to_csv(path+'training_data/standard/group/'+ str(group_id) + '/dev.txt',sep='\t',index=False,header=False)
  test_df.to_csv(path+'training_data/standard/group/'+ str(group_id) + '/test.txt',sep='\t',index=False,header=False)

  return train_df,dev_df,test_df


## 4. Divide the dataset into Groups/Classes

Here we iterate over the entire dataset to create group level datasets in the following steps :
  1. Iterate over the groups and read the full-dataset eveytime
  2. Get all the tags in the group from the **tag_group** lookup
  3. Iterate over training examples and see if the labels fall in the same group
  4. Remove training examples which don't belong to the group
  5. Create the training data format of the remaining dataset
  6. Split & Save the dataset

  ### TO DO :
    1. Make a single corpus for the entire dataset.


In [11]:
for grp_id in range(1,15):
  
  ## 1. Iterate over the groups and read the full-dataset eveytime
  print("=================================================================")
  print("Group ID being Processed : {}".format(grp_id))
  print("=================================================================")
  print("Reading Pickle File ...")

  question_tag = pd.read_pickle(path+data)

  ## 2. Get all the tags in the group from the **tag_group** lookup  

  group =  tag_group[tag_group.group == grp_id]
  labels = list(set(group['name']))

  ## 3. Iterate over training examples and see if the labels fall in the same group
  for index in tqdm(question_tag.index):
    tags = question_tag.loc[index,'Tag']
    group_tags = list()
    for tag in tags :
      if tag in labels :
        group_tags.append(tag)
    question_tag.at[index,'Tag'] =  group_tags
  print("Before Removal of Blank Data : {} ".format(question_tag.shape))

   ## 4. Remove training examples which don't belong to the group
  question_tag = question_tag[question_tag['Tag'].map(lambda d: len(d)) > 0]
  print("Final Data for Group ID  : {} is {}".format(grp_id,question_tag.shape))

  ## 5. Create the training data format of the remaining dataset
  training_data_format = create_training_format(question_tag)

  ## 6. Split & Save the dataset
  train_df,dev_df,test_df = create_splits(training_data_format,path,group_id=grp_id)

  print("=================================================================")



Group ID being Processed : 1
Reading Pickle File ...


100%|██████████| 1051992/1051992 [00:44<00:00, 23823.81it/s]


Before Removal of Blank Data : (1051992, 8) 


  2%|▏         | 3191/163030 [00:00<00:05, 31901.89it/s]

Final Data for Group ID  : 1 is (163030, 8)
Preparing training data format ...


...

Before Removal of Blank Data : (1051992, 8) 


 23%|██▎       | 3324/14291 [00:00<00:00, 33233.46it/s]

Final Data for Group ID  : 14 is (14291, 8)
Preparing training data format ...


100%|██████████| 14291/14291 [00:00<00:00, 31517.53it/s]


Cleaning Text ....
Cleaned Data Size : (14291, 2)
Splitting Training Data ... 
Training Dataset : 10003
Validation Dataset : 2144
Test Dataset : 2144
Path  : /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/standard/group/14/train.txt 


## 5. Create Corpus & Label Dictionary : Flair Corpus

For all the training splits created above for each group, we will be creating a corpus & vocabulary to train a different model.
  1. First we load the train, dev, test dataset and create corpus using ClassificationCorpus
  2. label dictionary is created using make_label_dictionary
  3. saving corpus and dictionary dumps for easy retrieval

In [14]:
path = '/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/standard/group/'

for grp in range(1,15):
  
  print("=================================================================")
  print("Group ID being Processed : {}".format(grp))
  print("=================================================================")
  
  # this is the folder in which train, test and dev files reside
  data_folder =path+str(grp)+'/'
  print(data_folder)

  print("Creating Corpus ...")
  # load corpus containing training, test and dev data
  corpus: Corpus = ClassificationCorpus(data_folder,
                                        test_file='test.txt',
                                        dev_file='dev.txt',
                                        train_file='train.txt')

  # 2. create the label dictionary
  label_dict = corpus.make_label_dictionary()

  print("Obtaining Corpus Statisitics...")
  stats  = corpus.obtain_statistics()
  json_stats = json.loads(stats)

  print(json_stats)
  
  with open(data_folder+'corpus_statistics.json', 'w') as f:
    json.dump(json_stats, f)

  print("Creating Dumps ... ")
  with open(data_folder+'classification_corpus.pkl',mode='wb') as f :
    pickle.dump(corpus,f)

  with open(data_folder + 'classification_corpus_label_dict.pkl',mode='wb') as f:
    pickle.dump(label_dict,f)

Group ID being Processed : 1
/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/standard/group/1/
Creating Corpus ...
2020-01-09 14:56:43,336 Reading data from /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/standard/group/1
2020-01-09 14:56:43,343 Train: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/standard/group/1/train.txt
2020-01-09 14:56:43,346 Dev: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/standard/group/1/dev.txt
2020-01-09 14:56:43,349 Test: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/standard/group/1/test.txt
2020-01-09 14:56:45,511 Computing label dictionary. Progress:


100%|██████████| 114121/114121 [05:56<00:00, 319.89it/s]

2020-01-09 15:02:42,757 [b'python', b'django', b'r', b'c', b'c++', b'matlab', b'qt', b'embedded', b'machine-learning', b'flask']


Obtaining Corpus Statisitics...
{'TRAIN': {'dataset': 'TRAIN', 'total_number_of_documents': 114121, 'number_of_documents_per_class': {'python': 45294, 'django': 8957, 'r': 11005, 'c': 16255, 'c++': 33272, 'matlab': 4553, 'qt': 3733, 'embedded': 421, 'machine-learning': 870, 'flask': 993}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 19629187, 'min': 9, 'max': 5701, 'avg': 172.00328598592722}}, 'TEST': {'dataset': 'TEST', 'total_number_of_documents': 24455, 'number_of_documents_per_class': {'r': 2425, 'c++': 7070, 'c': 3482, 'python': 9620, 'django': 1929, 'matlab': 990, 'machine-learning': 176, 'qt': 843, 'flask': 201, 'embedded': 96}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 4211200, 'min': 9, 'max': 3518, 'avg': 172.20200368022898}}, 'DEV': {'dataset': 'DEV', 'total_number_of_documents': 24454, 'number_of_documents_per_class': {'r': 2271, 'c': 3501, 'python': 9687, 'c++': 7249, 'django': 1932, 'matlab': 952, 'qt': 808, 'flask': 213, 'machine-lear

100%|██████████| 137377/137377 [06:42<00:00, 341.55it/s]

...

2020-01-09 17:42:37,646 [b'regex', b'perl']


Obtaining Corpus Statisitics...
{'TRAIN': {'dataset': 'TRAIN', 'total_number_of_documents': 13953, 'number_of_documents_per_class': {'regex': 10732, 'perl': 3657}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 1784337, 'min': 10, 'max': 2886, 'avg': 127.8819608686304}}, 'TEST': {'dataset': 'TEST', 'total_number_of_documents': 2990, 'number_of_documents_per_class': {'regex': 2277, 'perl': 800}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 381849, 'min': 19, 'max': 2374, 'avg': 127.70869565217392}}, 'DEV': {'dataset': 'DEV', 'total_number_of_documents': 2990, 'number_of_documents_per_class': {'regex': 2340, 'perl': 749}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 373761, 'min': 7, 'max': 2168, 'avg': 125.00367892976588}}}
Creating Dumps ... 
Group ID being Processed : 14
/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/standard/group/14/
Creating Corpus ...
2020-01-09 17:44:27,708 Reading data from /content/drive/My Drive/

100%|██████████| 10003/10003 [00:33<00:00, 302.18it/s]

2020-01-09 17:45:01,509 [b'excel', b'vba', b'excel-vba']


Obtaining Corpus Statisitics...
{'TRAIN': {'dataset': 'TRAIN', 'total_number_of_documents': 10003, 'number_of_documents_per_class': {'excel': 7261, 'vba': 4773, 'excel-vba': 3682}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 1741069, 'min': 10, 'max': 2934, 'avg': 174.05468359492153}}, 'TEST': {'dataset': 'TEST', 'total_number_of_documents': 2144, 'number_of_documents_per_class': {'excel': 1542, 'vba': 1087, 'excel-vba': 778}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 369777, 'min': 19, 'max': 2135, 'avg': 172.47061567164178}}, 'DEV': {'dataset': 'DEV', 'total_number_of_documents': 2144, 'number_of_documents_per_class': {'vba': 1032, 'excel-vba': 743, 'excel': 1581}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 368862, 'min': 14, 'max': 4458, 'avg': 172.04384328358208}}}
Creating Dumps ... 
